In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Forecasting Evaluation

- https://autogluon.github.io/fev/latest/tutorials/02-dataset-format/

Datasets are stored using Hugging Face `dataset` library in "fev format".

In [2]:
import warnings
import fev
import datasets
import numpy as np
import pandas as pd

warnings.simplefilter("ignore")
datasets.disable_progress_bars()

dataset_path = "/tmp/automl_ds.parquet"

/Users/khozzy/Projects/apytmiz/aptimyz-demand-forecast/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset preparation

In [3]:
df = pd.read_csv("../data/raw/train.csv")
df["series_id"] = df["store"].astype(str) + "_" + df["item"].astype(str)
df.drop(columns=["item", "store"], inplace=True)
df.head()

,date,sales,series_id
0,2013-01-01,13,1_1
1,2013-01-02,11,1_1
2,2013-01-03,14,1_1
3,2013-01-04,13,1_1
4,2013-01-05,10,1_1


In [4]:
ds = fev.utils.convert_long_df_to_hf_dataset(
    df, id_column="series_id", timestamp_column="date"
)
ds

Dataset({
    features: ['series_id', 'date', 'sales'],
    num_rows: 500
})

In [5]:
# verify if the dataset was converted correctly

# fev.utils.validate_time_series_dataset(
#     ds, id_column="series_id", timestamp_column="date"
# )

In [6]:
# save dataset to file
ds.to_parquet(dataset_path)

14615960

## Fetch eval data from batch tranform (S3)

In [7]:
eval_df = pd.read_csv(
    "s3://sagemaker-forecasting-aptimyz-output/transform/train.csv.out", header="infer"
)

# SageMaker Batch Transform creates a single output file by merging multiple CSV
# Some cleanup is needed for deduplication
eval_df = eval_df[eval_df["date"] != "date"]
eval_df.drop_duplicates(subset=["series_id", "date"], keep="first", inplace=True)

# Feature engineering
eval_df["series_id"] = eval_df["store"].astype(str) + "_" + eval_df["item"].astype(str)
eval_df["date"] = pd.to_datetime(eval_df["date"])
eval_df.drop(columns=["item", "store"], inplace=True)
eval_df[["p10", "p50", "p90", "mean"]] = eval_df[["p10", "p50", "p90", "mean"]].astype(
    np.float64
)

eval_df.head()

FileNotFoundError: sagemaker-forecasting-aptimyz-output/transform/train.csv.out

In [ ]:
eval_df.dtypes

date         datetime64[ns]
p10                 float64
p50                 float64
p90                 float64
mean                float64
series_id            object
dtype: object

## Evaluation task

### Metrics available
- Median estimation: MAE, WAPE, MASE
- Mean estimation: MSE, RMSE, RMSSE
- Logarithmic errors: RMSLE
- Percentage errors: MAPE, SMAPE
- Quantile loss: MQL, WQL, SQL

In [ ]:
task = fev.Task(
    dataset_path=dataset_path,
    horizon=7,
    num_windows=1,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column="series_id",
    timestamp_column="date",
    target="sales",
    eval_metric="RMSSE",
    extra_metrics=["WAPE", "WQL"],
)

In [ ]:
def model_forecast(window):
    past_data, future_data = window.get_input_data()
    future_dates = sorted(set(future_data[window.timestamp_column][0]))

    predictions = []
    for ts in past_data:
        filtered = eval_df[
            (eval_df[window.id_column] == ts[window.id_column])
            & (eval_df[window.timestamp_column].isin(future_dates))
        ].sort_values(window.timestamp_column)

        predictions.append(
            {
                "predictions": filtered["mean"].values.tolist(),
                "0.1": filtered["p10"].values.tolist(),
                "0.5": filtered["p50"].values.tolist(),
                "0.9": filtered["p90"].values.tolist(),
            }
        )

    return predictions


predictions_per_window = [model_forecast(window) for window in task.iter_windows()]

In [ ]:
eval_summary = task.evaluation_summary(
    predictions_per_window, model_name="sagemaker_automl", training_time_s=12
)

eval_summary

{'model_name': 'sagemaker_automl',
 'dataset_path': '/tmp/automl_ds.parquet',
 'dataset_config': None,
 'horizon': 7,
 'num_windows': 1,
 'initial_cutoff': -7,
 'window_step_size': 7,
 'min_context_length': 1,
 'max_context_length': None,
 'seasonality': 1,
 'eval_metric': 'RMSSE',
 'extra_metrics': ['WAPE', 'WQL'],
 'quantile_levels': [0.1, 0.5, 0.9],
 'id_column': 'series_id',
 'timestamp_column': 'date',
 'target': 'sales',
 'generate_univariate_targets_from': None,
 'known_dynamic_columns': [],
 'past_dynamic_columns': [],
 'static_columns': [],
 'task_name': '//tmp',
 'test_error': 0.6213560982790289,
 'training_time_s': 12,
 'inference_time_s': None,
 'dataset_fingerprint': '2b4ae952aefce86c',
 'trained_on_this_dataset': False,
 'fev_version': '0.6.0',
 'RMSSE': 0.6213560982790289,
 'WAPE': 0.1309864064592703,
 'WQL': 0.08558147103899073}

In [ ]:
from statsforecast.models import AutoETS, SeasonalNaive, Theta


def predict_with_model(
    task: fev.Task, model_name: str = "seasonal_naive"
) -> list[datasets.Dataset]:
    assert len(task.target_columns) == 1, "only univariate forecasting supported"
    if model_name == "seasonal_naive":
        model = SeasonalNaive(season_length=task.seasonality)
    elif model_name == "theta":
        model = Theta(season_length=task.seasonality)
    elif model_name == "ets":
        model = AutoETS(season_length=task.seasonality)
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    predictions_per_window = []
    for window in task.iter_windows():
        past_data, future_data = window.get_input_data()
        predictions = [
            {"predictions": model.forecast(y=ts[task.target], h=task.horizon)["mean"]}
            for ts in past_data
        ]
        predictions_per_window.append(datasets.Dataset.from_list(predictions))
    return predictions_per_window


o = predict_with_model(task, model_name="theta")

# import time
# import tqdm

# summaries = []
# for model_name in ["seasonal_naive", "ets", "theta"]:
#     start_time = time.time()
#     predictions_per_window = predict_with_model(task, model_name=model_name)
#     infer_time_s = time.time() - start_time
#     eval_summary = task.evaluation_summary(
#         predictions_per_window,
#         model_name=model_name,
#         inference_time_s=infer_time_s,
#         training_time_s=0.0,
#     )

#     summaries.append(eval_summary)


In [ ]:
o[0][0]

{'predictions': [21.806208382450748,
  21.808726128984627,
  21.811243875518507,
  21.81376162205239,
  21.816279368586272,
  21.81879711512015,
  21.82131486165403]}